In [1]:
import cv2
import numpy as np
import os
from scipy.ndimage import label

## Esto no funciona

In [3]:
def stylization_filter(image_path):
    
    img = cv2.imread(image_path)

    smoothed_image = cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)

    stylized_image = cv2.stylization(smoothed_image, sigma_s=60, sigma_r=0.07)

    cv2.imwrite('2_1.jpg', stylized_image)

stylization_filter('hoja.png')

## El resto si


In [4]:
def bgr_to_grayscale(bgr_image):
    # Extracting individual color channels
    blue_channel = bgr_image[:,:,0]
    green_channel = bgr_image[:,:,1]
    red_channel = bgr_image[:,:,2]

    # Applying the conversion equation
    grayscale_image = 0.114 * blue_channel + 0.587 * green_channel + 0.299 * red_channel

    # Converting to uint8 (8-bit) format
    grayscale_image = np.uint8(grayscale_image)

    return grayscale_image

# Read the stylized image obtained in the previous step
stylized_image = cv2.imread('2_1.jpg')

# Convert BGR to Grayscale
grayscale_image = bgr_to_grayscale(stylized_image)

# Save the images
cv2.imwrite('grayscale_image.jpg', grayscale_image)

True

In [5]:
# Read the grayscale image
grayscale_image = cv2.imread('grayscale_image.jpg', cv2.IMREAD_GRAYSCALE)

# Apply Gaussian Blur to the grayscale image with a 9x9 kernel and sigma = 1.5
blurred_image = cv2.GaussianBlur(grayscale_image, (9, 9), 1.5)

# Save the blurred image
cv2.imwrite('blurred_image.jpg', blurred_image)

True

In [6]:
# Read the blurred image obtained in the previous step
blurred_image = cv2.imread('blurred_image.jpg', cv2.IMREAD_GRAYSCALE)

# Define a 3x3 integer-valued kernel for dilation
kernel = np.ones((3, 3), np.uint8)

# Apply morphological dilation
dilated_image = cv2.dilate(blurred_image, kernel, iterations=1)

# Save the dilated image
cv2.imwrite('dilated_image.jpg', dilated_image)

True

In [7]:
# Read the dilated image obtained in the previous step
dilated_image = cv2.imread('dilated_image.jpg', cv2.IMREAD_GRAYSCALE)

# Define a 5x5 averaging filter kernel
kernel_size = 5
kernel = np.ones((kernel_size, kernel_size), np.float32) / (kernel_size * kernel_size)

# Apply 2D convolution using filter2D function
smoothed_image = cv2.filter2D(dilated_image, -1, kernel)

# Save the smoothed image
cv2.imwrite('smoothed_image.jpg', smoothed_image)

True

In [8]:
import cv2
import numpy as np

# Read the smoothed image obtained in the previous step
smoothed_image = cv2.imread('smoothed_image.jpg', cv2.IMREAD_GRAYSCALE)

# Apply Adaptive Thresholding
binary_threshold_image = cv2.adaptiveThreshold(
    smoothed_image, 
    maxValue=255, 
    adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
    thresholdType=cv2.THRESH_BINARY_INV,
    blockSize=11, 
    C=1
)

# Save the binary threshold image
cv2.imwrite('binary_threshold_image.jpg', binary_threshold_image)

True

In [9]:
# Read the dilated image obtained after Type Two processing
dilated_image_type_two = cv2.imread('dilated_image.jpg', cv2.IMREAD_GRAYSCALE)

# Apply Otsu's thresholding
_, otsu_thresholded = cv2.threshold(dilated_image_type_two, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Apply Binary thresholding as a fallback
_, binary_thresholded = cv2.threshold(dilated_image_type_two, 0, 255, cv2.THRESH_BINARY)

# Combine Otsu's and Binary thresholding
combined_thresholded = cv2.bitwise_and(otsu_thresholded, binary_thresholded)
end = cv2.bitwise_not(combined_thresholded)

# Save the combined thresholded image
cv2.imwrite('otsu_thresholded.jpg', otsu_thresholded)
cv2.imwrite('binary_thresholded.jpg', binary_thresholded)
cv2.imwrite('combined_thresholded_image.jpg', combined_thresholded)
cv2.imwrite('neg.jpg', end)

True

In [10]:
# Read the combined thresholded image obtained after background separation
combined_thresholded_image = cv2.imread('combined_thresholded_image.jpg', cv2.IMREAD_GRAYSCALE)

# Define a kernel for morphological operations
kernel_size = 3
kernel = np.ones((kernel_size, kernel_size), np.uint8)

# Perform erosion
erosion_result = cv2.erode(combined_thresholded_image, kernel, iterations=1)

# Perform dilation
dilation_result = cv2.dilate(combined_thresholded_image, kernel, iterations=1)

# Perform border extraction (subtract dilation result from erosion result)
border_extraction = cv2.absdiff(erosion_result, dilation_result)

# Save the border extraction result
cv2.imwrite('border_extraction_result.jpg', border_extraction)

True

In [11]:
# Read the binary threshold image
image1 = cv2.imread("binary_threshold_image.jpg", cv2.IMREAD_GRAYSCALE)

# Read the border extraction result
image2 = cv2.imread("border_extraction_result.jpg", cv2.IMREAD_GRAYSCALE)

# Find contours for image1
contours1, _ = cv2.findContours(image1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Find contours for image2
contours2, _ = cv2.findContours(image2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Draw the contours on the original images
contour_image1 = cv2.cvtColor(image1, cv2.COLOR_GRAY2BGR)
cv2.drawContours(contour_image1, contours1, -1, (0, 255, 0), 2)

contour_image2 = cv2.cvtColor(image2, cv2.COLOR_GRAY2BGR)
cv2.drawContours(contour_image2, contours2, -1, (0, 255, 0), 2)

# Save the images with contours drawn
cv2.imwrite("contour_image1.jpg", contour_image1)
cv2.imwrite("contour_image2.jpg", contour_image2)

True